# Sources
* Model architecture based on [Li et al., 2020](https://academic.oup.com/bioinformatics/article/36/4/1057/5578482?login=true), Repository available via download section on [Homepage](https://deepcleave.erc.monash.edu/)


In [1]:
import os
import sys
import csv
import math
import random
import numpy as np
from time import time
from tqdm import tqdm

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

from torchtext.vocab import build_vocab_from_iterator

from sklearn.metrics import roc_auc_score
from sklearn.model_selection import KFold

In [2]:
def seed_everything(seed: int):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True
    
seed_everything(1234)

In [3]:
def read_data(path):
    with open(path, "r") as csvfile:
        train_data = list(csv.reader(csvfile))[1:]  # skip col name
        sents, lbls = [], []
        for s, l in train_data:
            sents.append(s)
            lbls.append(l)
    return sents, lbls


def apply_random_masking(seq, num_tokens):
    """
    Mask `num_tokens` as 0 at random positions per sequence.
    """
    dist = torch.rand(seq.shape)
    m, _ = torch.topk(dist, num_tokens)
    return seq * (dist < m)


def regularized_auc(train_auc, dev_auc, threshold=0.0025):
    """
    Returns development AUC if overfitting is below threshold, otherwise 0.
    """
    return dev_auc if (train_auc - dev_auc) < threshold else 0


def save_metrics(*args, path):
    if not os.path.isfile(path):
        with open(path, "w", newline="\n") as f:
            f.write(
                ",".join(
                    [
                        "fold",
                        "epoch",
                        "train_loss",
                        "train_acc",
                        "train_auc",
                        "val_loss",
                        "val_acc",
                        "val_auc",
                    ]
                )
            )
            f.write("\n")
    if args:
        with open(path, "a", newline="\n") as f:
            f.write(",".join([str(arg) for arg in args]))
            f.write("\n")
            
def trainable_model_params(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

def total_model_params(model):
    return sum(p.numel() for p in model.parameters())

In [4]:
class CleavageDataset(Dataset):
    def __init__(self, seq, lbl):
        self.seq = seq
        self.lbl = lbl
    
    def __getitem__(self, idx):
        return self.seq[idx], self.lbl[idx]
    
    def __len__(self):
        return len(self.lbl)
    
class TrainBatch:
    def __init__(self, batch):
        ordered_batch = list(zip(*batch))
        seq = torch.tensor([encode_text(seq) for seq in ordered_batch[0]], dtype=torch.float)
        self.seq = apply_random_masking(seq, num_tokens=1)
        self.lbl = torch.tensor([int(l) for l in ordered_batch[1]], dtype=torch.float)
        
    def pin_memory(self):
        self.seq = self.seq.pin_memory()
        self.lbl = self.lbl.pin_memory()
        return self
    
def train_wrapper(batch):
    return TrainBatch(batch)


class EvalBatch:
    def __init__(self, batch):
        ordered_batch = list(zip(*batch))
        self.seq = torch.tensor([encode_text(seq) for seq in ordered_batch[0]], dtype=torch.float)
        self.lbl = torch.tensor([int(l) for l in ordered_batch[1]], dtype=torch.float)
        
    def pin_memory(self):
        self.seq = self.seq.pin_memory()
        self.lbl = self.lbl.pin_memory()
        return self
    
def eval_wrapper(batch):
    return EvalBatch(batch)

In [5]:
class Attention(nn.Module):
    def __init__(self, input_dim, hidden):
        super().__init__()

        self.W0 = nn.Parameter(
            nn.init.kaiming_normal_(torch.empty(input_dim, hidden)).to(device)
        )
        self.W = nn.Parameter(
            nn.init.kaiming_normal_(torch.empty(hidden, 1)).to(device)
        )
        self.b0 = nn.Parameter(
            nn.init.kaiming_normal_(torch.empty(hidden, 1)).squeeze().to(device)
        )
        self.b = nn.Parameter(
            nn.init.kaiming_normal_(torch.empty(1, 1)).squeeze(1).to(device)
        )

    def forward(self, x):
        # input shape: (batch_size, num_filters, seq_len)
        # input needs to be permuted
        energy = (
            x.permute(0, 2, 1) @ self.W0 + self.b0
        )  # linear activation, i.e. only identity

        # input shape: (batch_size, seq_len, hidden)
        energy = (energy @ self.W + self.b).squeeze()

        # input shape: (batch_size, seq_len)
        energy = F.softmax(energy, dim=1)

        # input shape: energy=(batch_size, seq_len)
        # output shape: (batch_size, input_dim)
        # batch-wise dot product along dims 1
        res = (energy.unsqueeze(1) @ x.permute(0, 2, 1)).squeeze(1)

        # input shape: (batch_size, input_dim)
        # output shape: (batch_size, input_dim+seq_len)
        return torch.concat([res, energy], axis=-1)

In [6]:
class CNNAttention(nn.Module):
    def __init__(
        self,
        seq_len,
        num_filters1,
        kernel_size1,
        num_filters2,
        kernel_size2a,
        kernel_size2b,
        kernel_size2c,
        num_filters3,
        kernel_size3a,
        kernel_size3b,
        kernel_size3c,
        attention_hidden1,
        attention_hidden2,
        hidden_size1,
        hidden_size2,
        dropout
    ):
        super().__init__()

        self.dropout = nn.Dropout(dropout)

        self.conv1 = nn.Conv1d(
            in_channels=1,
            out_channels=num_filters1,
            kernel_size=kernel_size1,
            padding="same",
        )

        self.conv2a = nn.Conv1d(
            in_channels=num_filters1,
            out_channels=num_filters2,
            kernel_size=kernel_size2a,
            padding="same",
        )

        self.conv2b = nn.Conv1d(
            in_channels=num_filters1,
            out_channels=num_filters2,
            kernel_size=kernel_size2b,
            padding="same",
        )

        self.conv2c = nn.Conv1d(
            in_channels=num_filters1,
            out_channels=num_filters2,
            kernel_size=kernel_size2c,
            padding="same",
        )

        self.conv3a = nn.Conv1d(
            in_channels=3 * num_filters2,
            out_channels=num_filters3,
            kernel_size=kernel_size3a,
            padding="same",
        )

        self.conv3b = nn.Conv1d(
            in_channels=3 * num_filters2,
            out_channels=num_filters3,
            kernel_size=kernel_size3b,
            padding="same",
        )

        self.conv3c = nn.Conv1d(
            in_channels=3 * num_filters2,
            out_channels=num_filters3,
            kernel_size=kernel_size3c,
            padding="same",
        )

        self.attention1 = Attention(input_dim=num_filters3, hidden=attention_hidden1)
        self.attention1r = Attention(input_dim=seq_len, hidden=attention_hidden2)

        self.attention2 = Attention(input_dim=num_filters3, hidden=attention_hidden1)
        self.attention2r = Attention(input_dim=seq_len, hidden=attention_hidden2)

        self.attention3 = Attention(input_dim=num_filters3, hidden=attention_hidden1)
        self.attention3r = Attention(input_dim=seq_len, hidden=attention_hidden2)

        self.fc1 = nn.Linear(3*num_filters3 + 3*seq_len, hidden_size1)
        self.fc2 = nn.Linear(hidden_size1, hidden_size2)
        self.fc3 = nn.Linear(hidden_size2, 1)

    def forward(self, seq):
        ### encoder ###

        # in: (batch_size, seq_len), out: (batch_size, num_filters1, seq_len)
        x1 = F.leaky_relu(self.dropout(self.conv1(seq.unsqueeze(1))))  # add dim=1 for conv

        # in: (batch_size, num_filters1, seq_len)
        # out: (batch_size, num_filters2, seq_len)
        y1 = F.leaky_relu(self.dropout(self.conv2a(x1)))
        y2 = F.leaky_relu(self.dropout(self.conv2b(x1)))
        y3 = F.leaky_relu(self.dropout(self.conv2c(x1)))
        # cat_y out: (batch_size, 3*num_filters2, seq_len)
        cat_y = self.dropout(torch.cat([y1, y2, y3], dim=1))

        # in: (batch_size, 3*num_filters2, seq_len)
        # out: (batch_size, num_filters3, seq_len)
        z1 = self.dropout(F.leaky_relu(self.conv3a(cat_y)))
        z2 = self.dropout(F.leaky_relu(self.conv3b(cat_y)))
        z3 = self.dropout(F.leaky_relu(self.conv3c(cat_y)))

        ### decoder ###

        # in: (batch_size, num_filters3, seq_len)
        # out w/out r, e.g. decoded_z1: (batch_size, num_filters3)
        # out w/ r: e.g. decoded_z1r: (batch_size, seq_len)
        decoded_z1 = self.attention1(z1)[:, : z1.shape[1]]
        decoded_z1r = self.attention1r(z1.permute(0, 2, 1))[:, : z1.shape[2]]

        decoded_z2 = self.attention2(z2)[:, : z2.shape[1]]
        decoded_z2r = self.attention2r(z2.permute(0, 2, 1))[:, : z2.shape[2]]

        decoded_z3 = self.attention3(z3)[:, : z3.shape[1]]
        decoded_z3r = self.attention3r(z3.permute(0, 2, 1))[:, : z3.shape[2]]

        # out: (batch_size, 3*num_filters3 + 3*seq_len)
        cat = self.dropout(
            torch.cat(
                [
                    decoded_z1,
                    decoded_z1r,
                    decoded_z2,
                    decoded_z2r,
                    decoded_z3,
                    decoded_z3r,
                ],
                dim=-1,
            )
        )

        # in: (batch_size, 3*num_filters3 + 3*seq_len), out: (batch_size, hidden_size1)
        out = self.dropout(F.leaky_relu(self.fc1(cat)))
        # in: (batch_size, hidden_size1), out: (batch_size, hidden_size2)
        out = self.dropout(F.leaky_relu(self.fc2(out)))
        # in: (batch_size, hidden_size2), out: (batch_size)
        return self.fc3(out).squeeze()

In [7]:
def process(model, loader, criterion, optim=None):
    epoch_loss, num_correct, total = 0, 0, 0
    preds, lbls = [], []
    
    for batch in tqdm(
        loader,
        desc="Train: " if optim is not None else "Eval: ",
        file=sys.stdout,
        unit="batches"
    ):
        seq, lbl = batch.seq, batch.lbl
        seq, lbl = seq.to(device), lbl.to(device)
        
        scores = model(seq)
        loss = criterion(scores, lbl)
        
        if optim is not None:
            optim.zero_grad()
            loss.backward()
            optim.step()
        
        epoch_loss += loss.item()
        num_correct += ((scores > 0) == lbl).sum().item()
        total += len(seq)
        preds.extend(scores.detach().tolist())
        lbls.extend(lbl.detach().tolist())
    return epoch_loss / total, num_correct / total, roc_auc_score(lbls, preds)

In [8]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
BATCH_SIZE = 512

# load train+dev set, mix it back as one
train_path = '../../data/c_train.csv'
dev_path = '../../data/c_val.csv'
test_path = '../../data/c_test.csv'

# combine previously split train and dev set
train_seqs, train_lbls = read_data(train_path)
dev_seqs, dev_lbls = read_data(dev_path)
total_seqs, total_lbls = np.array(train_seqs + dev_seqs), np.array(train_lbls + dev_lbls)

assert len(train_seqs) + len(dev_seqs) == len(total_seqs)
assert len(train_lbls) + len(dev_lbls) == len(total_lbls)

test_seqs, test_lbls = read_data(test_path)

test_data = CleavageDataset(test_seqs, test_lbls)
test_loader = DataLoader(test_data, batch_size=BATCH_SIZE, collate_fn=eval_wrapper, pin_memory=True, num_workers=10)

# create vocab from train seqs
vocab = build_vocab_from_iterator(train_seqs, specials=['<UNK>'])
vocab.set_default_index(vocab['<UNK>'])
encode_text = lambda x: vocab(list(x))

In [9]:
NUM_EPOCHS = 60
BATCH_SIZE = 512
VOCAB_SIZE = len(vocab)
LEARNING_RATE = 3e-4


params = {
    'seq_len': 10,
    'kernel_size1': 1,
    'kernel_size2a': 3,
    'kernel_size2b': 17,
    'kernel_size2c': 13,
    'kernel_size3a': 11,
    'kernel_size3b': 15,
    'kernel_size3c': 19,
    'num_filters1': 220,
    'num_filters2': 262,
    'num_filters3': 398,
    'attention_hidden1': 3,
    'attention_hidden2': 2,
    'hidden_size1': 79,
    'hidden_size2': 24,
    'dropout': 0.04,
}

criterion = nn.BCEWithLogitsLoss()

In [ ]:
kf = KFold(n_splits=10, shuffle=True, random_state=1234)
logging_path = "../../params/c_term/cnnAttention/results.csv"

start = time()
print("Starting Cross-Validation.")
highest_val_auc = 0

# get a new split
for fold, (train_idx, dev_idx) in enumerate(kf.split(total_seqs), 1):
    X_tr = total_seqs[train_idx]
    y_tr = total_lbls[train_idx]
    X_dev = total_seqs[dev_idx]
    y_dev = total_lbls[dev_idx]

    # create datasets and loads with current split
    train_data = CleavageDataset(X_tr, y_tr)
    train_loader = DataLoader(
        train_data,
        batch_size=BATCH_SIZE,
        shuffle=True,
        collate_fn=train_wrapper,
        pin_memory=True,
        num_workers=10,
    )

    dev_data = CleavageDataset(X_dev, y_dev)
    dev_loader = DataLoader(
        dev_data,
        batch_size=BATCH_SIZE,
        shuffle=True,
        collate_fn=eval_wrapper,
        pin_memory=True,
        num_workers=10,
    )

    # reset model weights with each new fold
    model = CNNAttention(**params).to(device)
    optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)

    # normal training loop
    for epoch in range(1, NUM_EPOCHS + 1):
        model.train()
        train_loss, train_acc, train_auc = process(
            model, train_loader, criterion, optimizer
        )

        model.eval()
        with torch.no_grad():
            val_loss, val_acc, val_auc = process(model, dev_loader, criterion)

        # save metrics
        save_metrics(
            fold,
            epoch,
            train_loss,
            train_acc,
            train_auc,
            val_loss,
            val_acc,
            val_auc,
            path=logging_path,
        )

        print(
            f"Training:   [Fold {fold:2d}, Epoch {epoch:2d}, Loss: {train_loss:8.6f}, Acc: {train_acc:.4f}, AUC: {train_auc:.4f}]"
        )
        print(f"Evaluation: [Fold {fold:2d}, Epoch {epoch:2d}, Loss: {val_loss:8.6f}, Acc: {val_acc:.4f}, AUC: {val_auc:.4f}]")
        
        reg_auc = regularized_auc(train_auc, val_auc, threshold=0)
        if reg_auc > highest_val_auc:
            highest_val_auc = reg_auc
            path = f"../../params/c_term/cnnAttention/auc{reg_auc:.4f}_fold{fold}_epoch{epoch}.pt"
            torch.save(model.state_dict(), path)

print("Finished Cross-Validation.")
train_time = (time() - start) / 60
print(f"Cross-Validation took {train_time} minutes.")

In [11]:
# load best model, evaluate on test set
best_model = sorted(
    [f for f in os.listdir("../../params/c_term/cnnAttention/") if f.endswith(".pt")],
    reverse=True,
)[0]
print("Loaded model: ", best_model)
model.load_state_dict(torch.load('../../params/c_term/cnnAttention/' + best_model))
model.eval()
test_loss, test_acc, test_auc = process(model, test_loader, criterion)
print(
    f"Test Set Performance: Loss: {test_loss:.6f}, Acc: {test_acc:.4f}, AUC: {test_auc:.4f}"
)

Loaded model:  auc0.8665_fold10_epoch60.pt
Eval: 100%|██████████████████████████████████████████████████████| 278/278 [00:01<00:00, 142.49batches/s]
Test Set Performance: Loss: 0.000635, Acc: 0.8574, AUC: 0.8666


In [11]:
best_model = sorted(
    [f for f in os.listdir("../../params/c_term/cnnAttention/") if f.endswith(".pt")],
    reverse=True,
)[0]
print("Loaded model: ", best_model)
model = CNNAttention(**params)
model.load_state_dict(torch.load('../../params/c_term/cnnAttention/' + best_model))
model.eval()

print(
    f"Total model params: {total_model_params(model)}, trainable model params: {trainable_model_params(model)}"
)

Loaded model:  auc0.8665_fold10_epoch60.pt
Total model params: 16084198, trainable model params: 16084198
